This notebook is created with the purpose of understanding the Jigsaw Puzzle using qwen2.5|0.5B. 

I am also using LoRA optimization technique with the purspose of understanding and ofcourse to make the model consume less computation resources.

I intend to understand the basics myself and share my learnings with fellow Kagglers. 

Please feel free to provide feedback, contribute to the understanding and correct me if need be(I hope Not:-))

If you really like the work - Please Upvote:-)




In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/qwen2.5/transformers/0.5b/1/config.json
/kaggle/input/qwen2.5/transformers/0.5b/1/merges.txt
/kaggle/input/qwen2.5/transformers/0.5b/1/LICENSE
/kaggle/input/qwen2.5/transformers/0.5b/1/README.md
/kaggle/input/qwen2.5/transformers/0.5b/1/tokenizer.json
/kaggle/input/qwen2.5/transformers/0.5b/1/vocab.json
/kaggle/input/qwen2.5/transformers/0.5b/1/tokenizer_config.json
/kaggle/input/qwen2.5/transformers/0.5b/1/model.safetensors
/kaggle/input/qwen2.5/transformers/0.5b/1/.gitattributes
/kaggle/input/qwen2.5/transformers/0.5b/1/generation_config.json
/kaggle/input/jigsaw-agile-community-rules/sample_submission.csv
/kaggle/input/jigsaw-agile-community-rules/train.csv
/kaggle/input/jigsaw-agile-community-rules/test.csv


### Setting Environment Variables

<ul>
    <li>TOKENIZERS_PARALLELISM controls whether the Hugging Face tokenizers library uses its own internal multi-threading to speed up tokenization.</li>
    <li>Reason to disable it is to prevent a conflict with PyTorch's multi-processing data loading, which can lead to deadlocks or severely degraded performance - 
</ul>

In [2]:
!pip install transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [3]:
import os, gc, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
    set_seed,
)

2025-09-09 19:37:12.390368: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757446632.739821      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757446632.846184      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
train_data = "/kaggle/input/jigsaw-agile-community-rules/train.csv"
model_id  = "/kaggle/input/qwen2.5/transformers/0.5b/1"

In [5]:
OUT_ROOT  = "/kaggle/working/qwen25_ce_headonly"
os.makedirs(OUT_ROOT, exist_ok=True)

In [6]:
SEED=42; 
N_FOLDS=5
MAX_LEN=512
EPOCHS=4
TRAIN_BATCH=4
GRAD_ACC=4
LR=2e-4
WARMUP=0.05
LABEL_SMOOTH = 0.0  # set 0.05 if you want mild smoothing

In [7]:
set_seed(SEED)
device = "cuda" if torch.cuda.is_available() else "cpu"

In [8]:
def _trim(x):
    x = "" if pd.isna(x) else str(x).strip()
    return x[:300] if len(x) > 0 else ""


In [9]:
def build_text_a(row):
    sub  = str(row["subreddit"])
    rule = str(row["rule"])
    pos1 = _trim(row.get("positive_example_1",""))
    neg1 = _trim(row.get("negative_example_1",""))
    parts = [f"r/{sub}", f"Rule: {rule}"]
    if pos1: parts.append(f"Yes: {pos1}")
    if neg1: parts.append(f"No: {neg1}")
    return " | ".join(parts)

In [10]:
def prepare_df(df):
    df = df.copy()
    df["text_a"] = df.apply(build_text_a, axis=1)
    df["text_b"] = df["body"].astype(str)
    return df

In [11]:
df = pd.read_csv(train_data)
df = prepare_df(df)

In [12]:
target  = df["rule_violation"].astype(int).values

In [13]:
# --------------- Tokenizer ---------------
tok = AutoTokenizer.from_pretrained(model_id, use_fast=True)
if tok.pad_token is None:
    if tok.eos_token is None:
        tok.add_special_tokens({"eos_token":"</s>"})
    tok.pad_token = tok.eos_token
    
tok.padding_side = "right"

In [14]:
collator = DataCollatorWithPadding(tokenizer=tok)

In [15]:
# --------------- Dataset ---------------
class PairDataset(Dataset):
    def __init__(self, df, labels=None, max_len=256):
        self.a = df["text_a"].tolist()
        self.b = df["text_b"].tolist()
        self.labels = labels
        self.max_len = max_len
    def __len__(self): return len(self.a)
    def __getitem__(self, i):
        item = tok(self.a[i], self.b[i], truncation=True, max_length=self.max_len)
        if self.labels is not None:
            # labels must be Long for CE
            item["labels"] = torch.tensor(int(self.labels[i]), dtype=torch.long)
        return item

In [16]:
# --------------- Model (head-only) ---------------
def build_headonly_model():
    model = AutoModelForSequenceClassification.from_pretrained(
        model_id,
        num_labels=2,
        trust_remote_code=True,
        torch_dtype=torch.float32,  # keep everything fp32 to avoid dtype mismatches
    )
    model.config.use_cache = False
    model.config.pad_token_id = tok.pad_token_id
    model.config.problem_type = "single_label_classification"

    # keep embeddings in sync if special tokens were added
    if model.get_input_embeddings().num_embeddings != len(tok):
        model.resize_token_embeddings(len(tok))

    # fresh classifier head
    hidden = model.config.hidden_size
    model.score = nn.Linear(hidden, 2)
    nn.init.xavier_uniform_(model.score.weight)
    nn.init.zeros_(model.score.bias)

    # freeze everything except the classifier head
    for n, p in model.named_parameters():
        p.requires_grad = ("score" in n)

    return model.to(device)

In [17]:
# --------------- Weighted CE & safe FP32 loss ---------------
pos_frac = float(target.mean())
neg_frac = 1.0 - pos_frac

# class_weights shape [2] on same device as model; weight(neg)=1, weight(pos)=neg/pos
cw = torch.tensor([1.0, (neg_frac / max(pos_frac, 1e-6))], dtype=torch.float32)

In [18]:
class FP32Trainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None, **kwargs):
        labels = inputs.get("labels")
        inputs = {k: v for k, v in inputs.items() if k != "labels"}
        outputs = model(**inputs)
        logits = outputs.logits.float()  # always compute loss in FP32

        # move class weights to correct device only here (model may be on GPU/CPU)
        loss_fct = nn.CrossEntropyLoss(
            weight=cw.to(logits.device),
            label_smoothing=LABEL_SMOOTH
        )
        loss = loss_fct(logits.view(-1, 2), labels.view(-1))

        return (loss, outputs) if return_outputs else loss

In [19]:
# --------------- CV Train ---------------
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
oof = np.zeros(len(df), dtype=float)

In [20]:
for fold, (trn_idx, val_idx) in enumerate(skf.split(df, target), 1):
    print(f"\n===== Fold {fold}/{N_FOLDS} =====")
    dtr = PairDataset(df.iloc[trn_idx], target[trn_idx], MAX_LEN)
    dvl = PairDataset(df.iloc[val_idx], target[val_idx], MAX_LEN)

    model = build_headonly_model()

    args = TrainingArguments(
        output_dir=f"{OUT_ROOT}/fold{fold}",
        num_train_epochs=EPOCHS,
        per_device_train_batch_size=TRAIN_BATCH,
        gradient_accumulation_steps=GRAD_ACC,
        learning_rate=LR,
        warmup_ratio=WARMUP,
        weight_decay=0.01,
        logging_steps=50,
        save_strategy="no",          # keep training simple (no checkpointing)
        report_to="none",
        remove_unused_columns=False,
        fp16=False, bf16=False,      # keep pure FP32 to avoid dtype issues
        dataloader_pin_memory=False,
        seed=SEED,
    )

    trainer = FP32Trainer(
        model=model,
        args=args,
        train_dataset=dtr,
        tokenizer=tok,
        data_collator=collator,
    )

    # quick sanity forward to ensure tensors/devices OK (no grads)
    tmp_loader = DataLoader(dtr, batch_size=2, shuffle=False, collate_fn=collator)
    batch = next(iter(tmp_loader))
    for k in batch: batch[k] = batch[k].to(device)
    with torch.no_grad():
        _ = model(**{k:v for k,v in batch.items() if k!="labels"})
    del tmp_loader, batch; gc.collect()

    trainer.train()

    # Save fold weights & tokenizer
    save_dir = f"{OUT_ROOT}/fold{fold}"
    os.makedirs(save_dir, exist_ok=True)
    trainer.model.save_pretrained(save_dir)
    tok.save_pretrained(save_dir)
    print(f"Saved fold {fold} to {save_dir}")

    # ---------- OOF AUC ----------
    model.eval()
    dl = DataLoader(dvl, batch_size=128, shuffle=False, collate_fn=collator)
    preds = []
    with torch.no_grad():
        for batch in dl:
            for k in batch: batch[k] = batch[k].to(device)
            logits = model(**{k:v for k,v in batch.items() if k!="labels"}).logits.float()
            logits = torch.nan_to_num(logits, nan=0.0, posinf=1e4, neginf=-1e4)
            prob = torch.softmax(logits, dim=1)[:, 1]
            prob = torch.nan_to_num(prob, nan=0.5)
            preds.append(prob.detach().cpu().numpy())

    prob1 = np.concatenate(preds)
    prob1 = np.clip(prob1, 0.0, 1.0)
    if np.isnan(prob1).any():
        prob1 = np.nan_to_num(prob1, nan=0.5)

    oof[val_idx] = prob1
    print(f"Fold {fold} AUC: {roc_auc_score(target[val_idx], prob1):.4f}")

    del trainer, model, dtr, dvl, dl, preds, prob1
    gc.collect()
    if torch.cuda.is_available(): torch.cuda.empty_cache()




===== Fold 1/5 =====


Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/qwen2.5/transformers/0.5b/1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
50,4.270700
100,1.943000
150,1.591800
200,1.537100


Saved fold 1 to /kaggle/working/qwen25_ce_headonly/fold1
Fold 1 AUC: 0.6948

===== Fold 2/5 =====


Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/qwen2.5/transformers/0.5b/1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Step,Training Loss
50,3.215000
100,2.009400
150,1.815900
200,1.685800


Saved fold 4 to /kaggle/working/qwen25_ce_headonly/fold4
Fold 4 AUC: 0.6096

===== Fold 5/5 =====


Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/qwen2.5/transformers/0.5b/1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
50,3.349400
100,2.138800
150,1.822200
200,1.676100


Saved fold 5 to /kaggle/working/qwen25_ce_headonly/fold5
Fold 5 AUC: 0.6464


In [21]:
print(f"\nOOF AUC: {roc_auc_score(target, oof):.4f}")
pd.DataFrame({"oof": oof, "y": target}).to_csv(f"{OUT_ROOT}/oof.csv", index=False)
print("Saved folds under:", OUT_ROOT)


OOF AUC: 0.6510
Saved folds under: /kaggle/working/qwen25_ce_headonly
